In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import utils

In [2]:
#train_data = pd.read_csv('austin_weather.csv')['TempAvgF']
train_data = pd.read_csv('../../Cleaning-the-data/tao-cleaned.csv')['Sea_Surface_Temp']
train_data = (train_data - train_data.mean())/train_data.std()
train_data = train_data.values[:10000]
print(train_data)
train_data.shape

[-0.75397095 -0.89195707 -1.24458829 ... -0.0742615  -0.04870851
 -0.08448269]


(10000,)

In [3]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence,self).__init__()
        self.lstm1 = nn.LSTM(1,32)
        self.lstm2 = nn.LSTM(32,32)
        self.lstm3 = nn.LSTM(32,1)
 
    def forward(self, seq):
        lstm1_out, _ = self.lstm1(seq, None)
        lstm2_out, _ = self.lstm2(lstm1_out, None)
        lstm3_out, _ = self.lstm3(lstm2_out, None)
        return lstm3_out.squeeze(1)

In [4]:
seq = Sequence()
criterion = nn.MSELoss()
optimizer = optim.Adam(seq.parameters(), lr=0.008)
bs = 32

In [5]:
for epoch in range(50):
    running_loss = 0
    running_error = 0
    num_batches = 0
    for i in range(0, 10000, 5):
        data = train_data[i:i+bs]
        xs = torch.Tensor(data[:-1]).view(-1,1,1)
        ys = torch.Tensor(data[1:])
        
        lstm_out = seq(xs)
        loss = criterion(lstm_out.view(-1), ys)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # compute some stats:
        
        num_batches+=1
        
        with torch.no_grad():
            running_loss += loss.item()
            error = utils.get_error(lstm_out.view(1,-1) , ys)
            running_error += error 

    # once the epoch is finished we divide the "running quantities"
    # by the number of batches
    if epoch % 10 == 0 : 
        total_loss = running_loss/num_batches
        total_error = running_error/num_batches

        print('epoch=',epoch, '\t loss=', total_loss, '\t error=', total_error*100 ,'percent')

epoch= 0 	 loss= 1.4949995321868264 	 error= 97.17019038837391 percent
epoch= 10 	 loss= 1.4227412087954616 	 error= 90.76863799283197 percent
epoch= 20 	 loss= 1.4182212708971147 	 error= 92.17928086344901 percent
epoch= 30 	 loss= 1.417236609641702 	 error= 90.5801747311828 percent
epoch= 40 	 loss= 1.4163511501215689 	 error= 90.70777484241749 percent
